# Extracting Results from First Run

For the first part, extract the results from the first run stored in Google drive, and appends them to create a consolidated view.

For the second part, it selects 100 instance ids to run based on the following criteria:
1. One of the SAST tools: Bandit, Semgrep or CodeQL, has a risk score > 0
2. Randomly select instance ids where code generation has completed for the remaining instances to fill up the 100 instance ids.

## Steps to Run
### Pre-requisites
1. You must use Google Colab to run this notebook and need to login with your SMU credentials.
2. You will need to add a shortcut folder in your Google Drive.
- After login, go to [Google Drive Folder: aisysevalteam-data](https://drive.google.com/drive/folders/18JLCTe-E6zvny5ZwJECLUz_sf0mBNAas)
- Click on aisysevalteam-data dropdown > Organise > Add Shortcut > All Locations > My Drive to add the aisysevalteam-data folder to your own Google Drive.
3. Export the `first 300.gsheet` to `first 300.csv`, removing unnecessary columns and making sure the foldername is correctly referenced.



In [1]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


### First part to consolidate results
Extract out the `first 300.csv` which stores the overall first run status.

In [2]:
import pandas as pd
import json

# Define the path to the Google Sheet file
file_path = '/content/drive/MyDrive/aisysevalteam-data/first 300.csv'

try:
    df = pd.read_csv(file_path)
    print("DataFrame loaded successfully:")
    display(df.head())
except FileNotFoundError:
    print(f"Error: File not found at {file_path}. Please make sure the file exists and Google Drive is mounted correctly.")
except Exception as e:
    print(f"An error occurred while reading the file: {e}")

# Identify rows where the 'link' column starts with 'https://drive'. This means the run was completed
drive_link_rows_df = df[df['link'].astype(str).str.startswith('https://drive')]

DataFrame loaded successfully:


,instance_id,who,link,foldername
0,astropy__astropy-12907,kimsia,https://drive.google.com/drive/folders/11eT6fW...,20251014_0922
1,astropy__astropy-14182,kimsia,https://drive.google.com/drive/folders/11eT6fW...,20251014_0922
2,astropy__astropy-14365,kimsia,https://drive.google.com/drive/folders/11eT6fW...,20251014_0922
3,astropy__astropy-14995,kimsia,https://drive.google.com/drive/folders/11eT6fW...,20251014_0922
4,astropy__astropy-6938,kimsia,https://drive.google.com/drive/folders/11eT6fW...,20251014_0922


Extract out the security_risk_score.json from each instance_id and foldername indicated in the `first 300.csv` file.

In [3]:
# Retrieve security scores
security_score_list = []

for index, row in drive_link_rows_df.iterrows():
    instance_id = row['instance_id']
    foldername = row['foldername']
    file_path = f"/content/drive/MyDrive/aisysevalteam-data/{foldername}/{instance_id}/run_1/security_risk_score.json"

    try:
        with open(file_path, 'r') as f:
            json_content = f.read()
            security_score_data = json.loads(json_content)
            security_score_data["foldername"] = foldername
            security_score_list.append(security_score_data)
    except FileNotFoundError:
        print(f"File not found for instance_id: {instance_id} and foldername: {foldername}")
    except json.JSONDecodeError:
        print(f"Error decoding JSON for instance_id: {instance_id} and foldername: {foldername}")

# Consolidate security scores into pandas DF
sec_scores_df_list = []

for score_data in security_score_list:
    if 'instance_id' in score_data and score_data.get('security_risk_score_result') is not None and 'tool_scores' in score_data['security_risk_score_result']:
        tool_scores = score_data['security_risk_score_result']['tool_scores']
        sec_scores_df_list.append({
            'instance_id': score_data['instance_id'],
            'foldername': score_data['foldername'],
            'bandit_score': tool_scores.get('bandit'),
            'semgrep_score': tool_scores.get('semgrep'),
            'codeql_score': tool_scores.get('codeql')
        })

sec_scores_df = pd.DataFrame(sec_scores_df_list)


File not found for instance_id: pallets__flask-5063 and foldername: 20251015_1250


Extract out the results.csv in each folder, and merge with the rows from `first 300.csv`

In [4]:
import os

data_dir = '/content/drive/MyDrive/aisysevalteam-data'
folder_list = [d for d in os.listdir(data_dir) if os.path.isdir(os.path.join(data_dir, d))]
all_results_dfs_list = []

for folder in folder_list:
    file_path = f"/content/drive/MyDrive/aisysevalteam-data/{folder}/results.csv"
    try:
        results_df = pd.read_csv(file_path)
        results_df["foldername"] = folder
        merged_df = pd.merge(drive_link_rows_df, results_df, on=['foldername', 'instance_id'], how='inner')
        all_results_dfs_list.append(merged_df)
    except FileNotFoundError:
        print(f"File not found for foldername: {foldername}")

combined_results_df = pd.concat(all_results_dfs_list).sort_values(by='instance_id').reset_index(drop=True)
# Merge combined_results_df with tool_scores_df with inner
merged_security_scores_n_results_df = pd.merge(combined_results_df, sec_scores_df, on=['instance_id', 'foldername'], how='inner')

# merged_security_scores_n_results_df.to_csv(f"{data_dir}/first_300_combined.csv")

### Second step to extract instance IDs for the final runs


In [5]:
# Filter out rows from merged_security_scores_n_results_df based on either one of security_risk_score, bandit_score, semgrep_score, codeql_score > 0
security_scores_gr_0_df = merged_security_scores_n_results_df[
    (merged_security_scores_n_results_df['security_risk_score'] > 0) |
    (merged_security_scores_n_results_df['bandit_score'] > 0) |
    (merged_security_scores_n_results_df['semgrep_score'] > 0) |
    (merged_security_scores_n_results_df['codeql_score'] > 0)
]

instance_ids_with_sec_scores = security_scores_gr_0_df['instance_id'].tolist()

# Filter rows with scores = 0 and randomly select the remaining instances
scores_0_df = merged_security_scores_n_results_df[~merged_security_scores_n_results_df['instance_id'].isin(instance_ids_with_sec_scores)]
sample_count = 100 - len(instance_ids_with_sec_scores)
sample_completed_run_df = scores_0_df[scores_0_df['generation_status'] == 'completed'].sample(n=sample_count, random_state=777)
final_instance_ids = instance_ids_with_sec_scores + sample_completed_run_df['instance_id'].tolist()


Copy the instance IDs to final_100.gsheet. Note that you will need to manually move it using the UI to aisysevalteam-data folder.

In [ ]:
!pip install gspread pandas google-auth


In [78]:
from google.colab import auth
auth.authenticate_user()

import gspread
from google.auth import default

# Authorize gspread using google-auth
creds, _ = default()
gc = gspread.authorize(creds)

# Convert the list to a pandas DataFrame and sort by instance_id
final_instance_ids_df = pd.DataFrame(final_instance_ids, columns=['instance_id']).sort_values(by='instance_id').reset_index(drop=True)

# Define the desired sheet name and target folder path for instructions
sheet_title = 'final_100' # Google Sheet titles typically don't include file extensions like .gsheet
target_folder_path = '/content/drive/MyDrive/aisysevalteam-data'

# Create a new Google Sheet in the root directory (gspread.create does not support creating in specific folders directly)
try:
    sh = gc.create(sheet_title)
    print(f"Created new Google Sheet: {sh.url}")
except Exception as e:
    print(f"Could not create new sheet ({sheet_title}), trying to open existing one: {e}")
    try:
        sh = gc.open(sheet_title)
        print(f"Opened existing Google Sheet: {sh.url}")
    except Exception as e_open:
         print(f"Could not open existing sheet either: {e_open}")
         sh = None


if sh:
    # Select the first worksheet
    worksheet = sh.sheet1

    # Clear any existing data
    worksheet.clear()

    # Convert the DataFrame to a list of lists (including header)
    values = [final_instance_ids_df.columns.tolist()] + final_instance_ids_df.values.tolist()

    # Write the data to the worksheet
    worksheet.update(values)

    print(f"DataFrame written to Google Sheet: {sh.url}")
    print(f"\nPlease manually move the sheet '{sheet_title}' from your Drive root to the folder '{target_folder_path}'")
else:
    print("Failed to create or open the Google Sheet.")

Created new Google Sheet: https://docs.google.com/spreadsheets/d/1gEwVhRcfJZKXV2_o7Bw2SQmrBjaoTWNKouYo0nduvDE
DataFrame written to Google Sheet: https://docs.google.com/spreadsheets/d/1gEwVhRcfJZKXV2_o7Bw2SQmrBjaoTWNKouYo0nduvDE

Please manually move the sheet 'final_100' from your Drive root to the folder '/content/drive/MyDrive/aisysevalteam-data'
